In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

sc = spark.sparkContext

In [3]:
filepath = "file:////home/talentum/mishika/data/15stocks_price.csv"

df = spark.read.option("header",True).csv(filepath)
df.show()
df.printSchema()

+------------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+------+-----+-----+-----+
|      date_gsheets| AAPL|   KO| NFLX|BRK.B|  DIS|   IBM|   VZ|  WMT|   GE| TSLA|   MA|  AMZN| MSFT|   UN|    V|
+------------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+------+-----+-----+-----+
|  01/03/2012 16:00|58.75|35.07|10.32|77.68|38.31| 186.3|39.73|60.33|18.36|28.08|36.84|179.03|26.77|34.92|25.75|
|  01/04/2012 16:00|59.06|34.85|11.49| 76.8|38.85|185.54|39.21|59.71|18.56|27.71|35.63|177.51| 27.4|34.45|25.29|
|  01/05/2012 16:00|59.72|34.69|11.33|76.93| 39.5|184.66|38.94|59.42|18.55|27.12|35.24|177.61|27.68|34.49|25.48|
|  01/06/2012 16:00|60.34|34.47|12.33|76.39|39.91|182.54|38.33|   59|18.65|26.91|34.29|182.61|28.11|33.27|25.18|
|  01/09/2012 16:00|60.25|34.47|14.03|76.29|39.75|181.59|38.37|59.18|18.86|27.25|34.58|178.56|27.74|33.72|24.98|
|  01/10/2012 16:00|60.46|34.67|13.69|77.42|39.63|181.31|38.57|59.04|18.72|27.62|34.88|179.34|27

In [5]:
from pyspark.sql.functions import *
df = df.withColumn("date_gsheets",to_date(df.date_gsheets,"MM/dd/yyyy HH:mm"))
df = df.withColumnRenamed('BRK.B', 'BRK')
df.show()

+------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+------+-----+-----+-----+
|date_gsheets| AAPL|   KO| NFLX|  BRK|  DIS|   IBM|   VZ|  WMT|   GE| TSLA|   MA|  AMZN| MSFT|   UN|    V|
+------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+------+-----+-----+-----+
|  2012-01-03|58.75|35.07|10.32|77.68|38.31| 186.3|39.73|60.33|18.36|28.08|36.84|179.03|26.77|34.92|25.75|
|  2012-01-04|59.06|34.85|11.49| 76.8|38.85|185.54|39.21|59.71|18.56|27.71|35.63|177.51| 27.4|34.45|25.29|
|  2012-01-05|59.72|34.69|11.33|76.93| 39.5|184.66|38.94|59.42|18.55|27.12|35.24|177.61|27.68|34.49|25.48|
|  2012-01-06|60.34|34.47|12.33|76.39|39.91|182.54|38.33|   59|18.65|26.91|34.29|182.61|28.11|33.27|25.18|
|  2012-01-09|60.25|34.47|14.03|76.29|39.75|181.59|38.37|59.18|18.86|27.25|34.58|178.56|27.74|33.72|24.98|
|  2012-01-10|60.46|34.67|13.69|77.42|39.63|181.31|38.57|59.04|18.72|27.62|34.88|179.34|27.84|33.91|24.95|
|  2012-01-11|60.36|34.03|13.16|   78

In [6]:
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import functions as F

return_df=df
my_window = Window.partitionBy().orderBy("date_gsheets")
for col_name in return_df.columns:
    if col_name!='date_gsheets':
     return_df = return_df.withColumn(col_name, F.when(F.isnull(col(col_name) - F.lag(col(col_name)).over(my_window)), 0)
                                      .otherwise(col(col_name) - F.lag(col(col_name)).over(my_window))/F.lag(col(col_name)).over(my_window))
for c in return_df.columns:
    if c!='date_gsheets':
        return_df = return_df.withColumn(c, F.round(c, 6))

return_df=return_df.dropna()
return_df.show(5)

+------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|date_gsheets|     AAPL|       KO|     NFLX|      BRK|      DIS|      IBM|       VZ|      WMT|       GE|     TSLA|       MA|     AMZN|     MSFT|       UN|        V|
+------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  2012-01-04| 0.005277|-0.006273| 0.113372|-0.011329| 0.014096|-0.004079|-0.013088|-0.010277| 0.010893|-0.013177|-0.032845| -0.00849| 0.023534|-0.013459|-0.017864|
|  2012-01-05| 0.011175|-0.004591|-0.013925| 0.001693| 0.016731|-0.004743|-0.006886|-0.004857| -5.39E-4|-0.021292|-0.010946|  5.63E-4| 0.010219| 0.001161| 0.007513|
|  2012-01-06| 0.010382|-0.006342| 0.088261|-0.007019|  0.01038|-0.011481|-0.015665|-0.007068| 0.005391|-0.007743|-0.026958| 0.028152| 0.015535|-0.035373|-0.011774|
|  2012-01

In [7]:
import pyspark.sql.functions as F

#cov_aapl_ko = return_df.stat.cov('AAPL', 'KO')
#cov_aapl_ko

In [8]:
#std_aapl = return_df.agg(F.stddev('AAPL')).collect()[0][0]
#print(std_aapl)
#std_ko = return_df.agg(F.stddev('KO')).collect()[0][0]
#print(std_ko)

In [9]:
correlation_aapl_ko = return_df.stat.corr("AAPL","KO")
correlation_aapl_ko

0.20541728655839

In [10]:
return_df=return_df.drop("date_gsheets")
return_df.show(5)

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|     AAPL|       KO|     NFLX|      BRK|      DIS|      IBM|       VZ|      WMT|       GE|     TSLA|       MA|     AMZN|     MSFT|       UN|        V|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| 0.005277|-0.006273| 0.113372|-0.011329| 0.014096|-0.004079|-0.013088|-0.010277| 0.010893|-0.013177|-0.032845| -0.00849| 0.023534|-0.013459|-0.017864|
| 0.011175|-0.004591|-0.013925| 0.001693| 0.016731|-0.004743|-0.006886|-0.004857| -5.39E-4|-0.021292|-0.010946|  5.63E-4| 0.010219| 0.001161| 0.007513|
| 0.010382|-0.006342| 0.088261|-0.007019|  0.01038|-0.011481|-0.015665|-0.007068| 0.005391|-0.007743|-0.026958| 0.028152| 0.015535|-0.035373|-0.011774|
|-0.001492|      0.0| 0.137875|-0.001309|-0.004009|-0.005204| 0.001044| 0.003051|  0.011

In [11]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=return_df.columns, outputCol=vector_col)
return_df_vector = assembler.transform(return_df).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(return_df_vector, vector_col)


In [12]:
matrix = Correlation.corr(return_df_vector, vector_col).collect()[0][0]
corrmatrix = matrix.toArray().tolist()

df = spark.createDataFrame(corrmatrix,return_df.columns)

for c in df.columns:
    df = df.withColumn(c,F.round(c,2))

df.show()

+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|AAPL|  KO|NFLX| BRK| DIS| IBM|  VZ| WMT|  GE|TSLA|  MA|AMZN|MSFT|  UN|   V|
+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| 1.0|0.21|0.12|0.31|0.27|0.26|0.17|0.17|0.26| 0.2|0.35|0.26|0.33|0.25| 0.3|
|0.21| 1.0|0.09|0.47|0.37|0.32|0.37|0.31|0.35|0.14|0.36|0.23|0.32|0.41|0.33|
|0.12|0.09| 1.0|0.19|0.15|0.12|0.04| 0.1|0.16|0.23|0.25| 0.3|0.21|0.15|0.22|
|0.31|0.47|0.19| 1.0|0.54|0.48|0.42|0.33|0.55|0.22|0.53|0.33|0.43|0.42| 0.5|
|0.27|0.37|0.15|0.54| 1.0|0.34|0.34|0.27|0.43|0.23|0.43|0.31|0.35|0.33|0.41|
|0.26|0.32|0.12|0.48|0.34| 1.0| 0.3|0.23|0.43|0.18|0.38|0.23|0.39| 0.3|0.36|
|0.17|0.37|0.04|0.42|0.34| 0.3| 1.0|0.29|0.37|0.11|0.28|0.17|0.29|0.31|0.27|
|0.17|0.31| 0.1|0.33|0.27|0.23|0.29| 1.0|0.24|0.11|0.25|0.14|0.22|0.23|0.25|
|0.26|0.35|0.16|0.55|0.43|0.43|0.37|0.24| 1.0|0.19| 0.4|0.26|0.34|0.33|0.38|
| 0.2|0.14|0.23|0.22|0.23|0.18|0.11|0.11|0.19| 1.0|0.27|0.26|0.21|0.18|0.23|

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Rerun this line before running "plot_my_scatter()"
%matplotlib

# Plotting more than 20 images? You may want to uncomment the code below:
#import warnings
#warnings.filterwarnings('ignore')

Using matplotlib backend: <object object at 0x0000023DCA90D610>


In [2]:
def get_port_weights():
    
    # Weights of Arlington == 1 through to 0 in decrements of 0.1
    weights_altn = list(np.round(np.arange(1, -0.1, -0.1), 2))
    
    # Weights of Brighton == 0 through to 1 in increments of 0.1
    weights_bton = weights_altn[:]
    weights_bton.reverse()
    
    return weights_altn, weights_bton

In [3]:
get_port_weights()

([1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0],
 [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

In [4]:
def get_port_returns_risk(expected_returns, total_risks, correlation, weights):
    
    
    port_returns = []
    port_risks = []
    for i in range(len(weights[0])):
        # Asset Weights
        w_altn = weights[0][i]
        w_bton = weights[1][i]
        
        # Asset Expected returns
        exp_r_altn = expected_returns[0]
        exp_r_bton = expected_returns[1]
        
        # Asset Risks
        risk_altn = total_risks[0]
        risk_bton = total_risks[1]
        
        # Portfolio Return
        port_ret = w_altn * exp_r_altn + w_bton * exp_r_bton
        
        # Portfolio Risk
        port_risk = np.sqrt((w_altn ** 2) * (risk_altn ** 2) \
                                + (w_bton ** 2) * (risk_bton ** 2) \
                                    + (2 * w_altn * w_bton * risk_altn * risk_bton * correlation))

        port_returns.append(port_ret)
        port_risks.append(port_risk)
    
    return port_returns, port_risks

In [5]:
def plot_my_scatter(expected_returns, total_risks, weights, correlation):

    port_returns, port_risks = get_port_returns_risk(expected_returns, total_risks, correlation, weights)
    
    plt.figure(figsize=(12, 8))
    plt.title(f"Portfolio Return vs. Portfolio Risk when ρ = {correlation}", fontsize=18)
    plt.scatter(port_risks, port_returns, color='#ffbd4a', zorder=2)
    plt.plot(port_risks, port_returns, color='#39b8eb', zorder=1, linewidth=2)
    plt.xlabel("Portfolio Risk (σ)")
    plt.ylabel("Expected Portfolio Return (E[r])")
    plt.ion()
    
    return

In [6]:
# Stock parameters
expected_r_arlington = .12
expected_r_brighton = .18
risk_arlington = .2 
risk_brighton = .3

In [7]:
%matplotlib

Using matplotlib backend: QtAgg


In [8]:
plot_my_scatter(expected_returns=[expected_r_arlington,
                                  expected_r_brighton],
               total_risks=[risk_arlington,
                           risk_brighton],
               weights=get_port_weights(),
               correlation=.3)